In [14]:
from dataclasses import dataclass, field
from __future__ import annotations

In [ ]:
class Calculator:
    def __init__(self) -> None:
        # Ordered according to BIDMAS
        self.operators: dict[str, callable] = {
            "^": lambda x, y: x ** y,
            "/": lambda x, y: x / y,
            "//": lambda x, y: x // y,
            "%": lambda x, y: x % y,
            "*": lambda x, y: x * y,
            "+": lambda x, y: x + y,
            "-": lambda x, y: x - y,
        }
        return

    def contains_operator(self, expression: str) -> dict[str, bool | str]:

        for operator, _ in self.operators.items():
            if operator in expression:
                return {
                    "operator_found": True,
                    "operator": operator
                }
        return {
            "operator_found": False,
            "operator": ""
        }

    def split(self, expression: str) -> str:

        expression_data = self.contains_operator(expression)
        if not expression_data[0]:
            return expression

        operator_index = expression.index(expression_data[1])
        left_hand_side: str = expression[:operator_index]
        right_hand_side: str = expression[operator_index+1:]

        return {
            "x": left_hand_side,
            "y": right_hand_side,
            "operator": expression_data[1],
        }

    def evaluate(self, expression: str) -> float:
        if expression[0].lower() == "f":
            return


calc = Calculator()

expression = "1+1+1"


calc.evaluate(expression=expression)

('1',
 {'x': '1',
  'y': '1',
  'operator': '+',
  'operation': <function __main__.Calculator.contains_operator.<locals>.<lambda>(x, y)>})

In [3]:
# Ordered according to BIDMAS
OPERATORS: dict[str, callable] = {
    "^": lambda x, y: x ** y,
    "/": lambda x, y: x / y,
    "//": lambda x, y: x // y,
    "%": lambda x, y: x % y,
    "*": lambda x, y: x * y,
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
}

In [125]:
@dataclass
class Node:
    expression: str = field(default="0")
    parent_node: Node = field(default=None)

    value: float = field(default=0, init=False)
    is_numeric: bool = field(default=False, init=False)
    operator: str = field(default="", init=False)
    operation: callable = field(default=None, init=False)
    children: tuple[Node] = field(default_factory=tuple, init=False)

    def __post_init__(self) -> None:
        self.operator = self.contained_operator(self.expression)
        self.operation = OPERATORS.get(self.operator)
        try:
            self.value = float(self.expression)

            self.is_numeric = True
        except ValueError:
            self.is_numeric = False
        except Exception as e:
            raise e

    def __repr__(self) -> str:
        if self.children:
            return f"<Node: {self.operator.center(3).join([child.expression for child in self.children])}>"

        return f"<Leaf Node: {self.expression}>"

    @staticmethod
    def contained_operator(expression: str) -> str:
        for operator, _ in OPERATORS.items():
            if operator in expression:
                return operator

        return ""

    def set_operator(self, operator: str) -> None:
        if not operator in OPERATORS.keys():
            raise NameError(f"\"{operator}\" operator not defined")

        self.operator = operator
        self.operation = OPERATORS.get(operator)

In [135]:
@dataclass
class Tree:
    initial_expression: str = field(repr=False)
    nodes: list = field(default_factory=list, init=False)

    def __post_init__(self) -> None:
        self.nodes.append(Node(self.initial_expression))
        self.propagate_single_node(self.nodes[0])

    def __getitem__(self, key) -> Node:
        return self.nodes[key]

    def __setitem__(self, key, value):
        self.nodes[key] = value

    def __delitem__(self, key):
        del self.nodes[key]

    def __repr__(self) -> str:
        def traverse(node: Node) -> str:
            if not node.children:
                return f"{node}"
            children = [traverse(child) for child in node.children]
            return f"{node.expression} (\n{', '.join(children)}\n)"

        return traverse(self.root)

    @property
    def root(self) -> Node:
        return self.nodes[0]

    def propagate_single_node(self, prev_node: Node) -> Node:
        expression = prev_node.expression
        if prev_node.operator:
            operator_index = expression.index(prev_node.operator)
            side_a: str = expression[:operator_index]
            side_b: str = expression[operator_index+1:]

            new_node_a = Node(side_a, prev_node)
            new_node_b = Node(side_b, prev_node)
            prev_node.children = (new_node_a, new_node_b)

            self.nodes.append(new_node_a)
            self.nodes.append(new_node_b)

            return prev_node

    def propagate_tree(self) -> None:
        def traverse(node: Node):
            if not node.children:
                self.propagate_single_node(node)
                # After propagating, node now has children (if it was an operator)
                for child in node.children:
                    traverse(child)
            else:
                for child in node.children:
                    traverse(child)
            return node

        return traverse(self.root)

In [136]:
tree = Tree("1/1+1*1+1")
tree.propagate_tree()
tree

1/1+1*1+1 (
<Leaf Node: 1>, 1+1*1+1 (
1+1 (
<Leaf Node: 1>, <Leaf Node: 1>
), 1+1 (
<Leaf Node: 1>, <Leaf Node: 1>
)
)
)